In [69]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Renaming of files

In [70]:
# Path to the folder containing CSV files
folder_path = 'Datatest'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Iterate through each CSV file and rename them
for index, csv_file in enumerate(csv_files, start=1):
    # Generate the new file name
    new_file_name = f'subject_{index}.csv'
    # Build the full path to the CSV file
    old_file_path = os.path.join(folder_path, csv_file)
    new_file_path = os.path.join(folder_path, new_file_name)
    # Rename the file
    os.rename(old_file_path, new_file_path)
    print(f'Renamed {csv_file} to {new_file_name}')

Renamed subject_1.csv to subject_1.csv


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Datatest\\subject_10.csv' -> 'Datatest\\subject_2.csv'

# Making the name of coloumn uniform

In [71]:
for i in range(1, 75):
    df = pd.read_csv(f'Datatest/subject_{i}.csv')
    
    # If a column is called "WRITING.TEXT", rename it to "WRITING.TEXT.Element:Text"
    if "WRITING.TEXT" in df.columns:
        df.rename(columns={"WRITING.TEXT": "WRITING.TEXT.Element:Text"}, inplace=True)
        df.to_csv(f'Datatest/subject_{i}.csv', index=False)  # Save the modified DataFrame
        print(f"Renamed 'WRITING.TEXT' to 'WRITING.TEXT.Element:Text' in subject_{i}.csv")
    else:
        print(f"No 'WRITING.TEXT' column found in subject_{i}.csv")

No 'WRITING.TEXT' column found in subject_1.csv
No 'WRITING.TEXT' column found in subject_2.csv
No 'WRITING.TEXT' column found in subject_3.csv
No 'WRITING.TEXT' column found in subject_4.csv
No 'WRITING.TEXT' column found in subject_5.csv
No 'WRITING.TEXT' column found in subject_6.csv
No 'WRITING.TEXT' column found in subject_7.csv
No 'WRITING.TEXT' column found in subject_8.csv
No 'WRITING.TEXT' column found in subject_9.csv
No 'WRITING.TEXT' column found in subject_10.csv
No 'WRITING.TEXT' column found in subject_11.csv
No 'WRITING.TEXT' column found in subject_12.csv
No 'WRITING.TEXT' column found in subject_13.csv
No 'WRITING.TEXT' column found in subject_14.csv
No 'WRITING.TEXT' column found in subject_15.csv
No 'WRITING.TEXT' column found in subject_16.csv
No 'WRITING.TEXT' column found in subject_17.csv
No 'WRITING.TEXT' column found in subject_18.csv
No 'WRITING.TEXT' column found in subject_19.csv
No 'WRITING.TEXT' column found in subject_20.csv
No 'WRITING.TEXT' column foun

In [72]:
df = pd.read_csv('Datatest/subject_53.csv')
df.columns

Index(['ID', 'WRITING.TITLE', 'WRITING.DATE', 'WRITING.INFO',
       'WRITING.TEXT.Element:Text'],
      dtype='object')

In [73]:
df.head()

,ID,WRITING.TITLE,WRITING.DATE,WRITING.INFO,WRITING.TEXT.Element:Text
0,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:32:12',reddit post,b'Yeah'
1,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:32:05',reddit post,b'Ok thanks \xf0\x9f\x99\x8c'
2,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:30:14',reddit post,b'It\xe2\x80\x99s not the it\xe2\x80\x99s sti...
3,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:23:03',reddit post,b'Give me one of your worsts then'
4,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:18:47',reddit post,b'\xf0\x9f\x98\xad'


# Removing the b and first and last apostrophes

In [74]:
df.describe()

,ID,WRITING.TITLE,WRITING.DATE,WRITING.INFO,WRITING.TEXT.Element:Text
count,875,875,875,875,875
unique,1,22,875,1,838
top,eRisk2023-T3_Subject31,b' ',b'2022-10-18 15:32:12',reddit post,b'Real'
freq,875,854,1,875,10


In [75]:
for index, row in df.iterrows():

    text_value = row['WRITING.TEXT.Element:Text']
    if isinstance(text_value, str) and text_value[0] == ' ' and text_value[1] == 'b' and text_value[2] == "'" and text_value[-1] == ' ' and text_value[-2] == "'":
        df.at[index, 'WRITING.TEXT.Element:Text'] = text_value[3:-2]
        #print the new value at that index
        print(df.at[index, 'WRITING.TEXT.Element:Text'])
        
    elif isinstance(text_value, float) and np.isnan(text_value):
        # Handle NaN values if needed
        pass

Yeah
Ok thanks \xf0\x9f\x99\x8c
It\xe2\x80\x99s not the it\xe2\x80\x99s still dark
Give me one of your worsts then
\xf0\x9f\x98\xad
Your ass too vanilla anyways
I\xe2\x80\x99m really really bored
Cause I\xe2\x80\x99m God
Sure
It\xe2\x80\x99s been up \xf0\x9f\x98\xad
I\xe2\x80\x99m 19 do it
Very
In due time brother, in due time \xf0\x9f\x98\x8c
U really do
Making a post about u
It was a joke jeez
Ouch \xf0\x9f\x98\xad
Like spare a whore a dollar will ya \xf0\x9f\x98\x95
Thanks \xf0\x9f\xa5\xb0
My What-The-Fuck-O-Meter\n\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\n\n-Jeffery Dahmer wasn\xe2\x80\x99t as bad as u\n\n-Honestly you deserve to be locked up\n\n-EW \n\n-Bitch clear your cache and history jesus\n\n-Not VPN worthy but ew\n\n-That\xe2\x80\x99s like what old people find obscene\n\n-My dog likes worse shit\n\n-Bitch that\xe2\x80\x99s no

# Removing unicode characters

In [76]:
for index, row in df.iterrows():

    text_value = row['WRITING.TEXT.Element:Text']

   # print (text_value)
    # if the string contains a substr of \xe2\x80\x99 replace it with '

    if isinstance(text_value, str):
        if '\\xe2\\x80\\x99' in text_value:
            print(text_value.count('\\xe2\\x80\\x99'))
        
    elif isinstance(text_value, float) and np.isnan(text_value):
        # Handle NaN values if needed
        pass

2
1
1
1
1
3
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
3
2
1
2
1
1
1
1
1
1
1
1
1
1
1
4
1
1
1
1
1
6
3
1
1
1
1
1
2
2
1
1
2
1
1
1
3
1
2
5
1
3
1
1
1
1
2
2
1
1
2
1
1
3
1
1
1
3
1
1
3
2
1
1
2
1
2
1
1
1
2
2
1
1
4
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
2
2
1
1
3
1
1
5
1
1
1
1
1
1
2
1
1
2
1
2
1
2
2
1
1
2
2
3
1
3
1
2
2
2
1
1
2
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
4
3
1
4
1
1
1
1
1
1
2
1
1
1
3
1
8
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
2
1
1
2
5
1
2
2
1
1
1
1
1
1
4
1
2
1
3
1
5
1
2
2
4
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
2
1
3
3
1
2
1
1
1
1
1
1
1
2
1
3
1
2
1
1
1
1
4
2
1
1
1
1
2
2
2
2
1
1
5
1
2
1
3
1
1
1
1
2
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1


In [77]:
for index, row in df.iterrows():

    text_value = row['WRITING.TEXT.Element:Text']
    if isinstance(text_value, str) and '\\xe2\\x80\\x99' in text_value:
        #print("YEMS")
        df.at[index, 'WRITING.TEXT.Element:Text'] = text_value.replace('\\xe2\\x80\\x99', "'")

    if isinstance(text_value, str) and '\\xe2\\x80\\x94' in text_value:
        df.at[index, 'WRITING.TEXT.Element:Text'] = text_value.replace('\\xe2\\x80\\x94', "-")
        
    # if isinstance(text_value, str) and '\\xf0\\x9f\\x98' in text_value:
    #     df.at[index, 'WRITING.TEXT.Element:Text'] = text_value.replace('\\xf0\\x9f\\x98', "")

    # if isinstance(text_value, str) and '\\xf0\\x9f\\x98' in text_value:
    #     df.at[index, 'WRITING.TEXT.Element:Text'] = text_value.replace('\\xf0\\x9f\\x92', "")
    
    if isinstance(text_value, str) and '\\n' in text_value:
        df.at[index, 'WRITING.TEXT.Element:Text'] = text_value.replace('\\n', " ")
        #print("YEMS")
    
    if isinstance(text_value, str):
        # Define the regular expression pattern to match words starting with \xf0\x9f
        pattern = r'\\xf0\\x9f\S*'
        # Replace matched words with a single space
        replaced_text = re.sub(pattern, ' ', text_value)
        # Update the DataFrame with the replaced text
        df.at[index, 'WRITING.TEXT.Element:Text'] = replaced_text

    elif isinstance(text_value, float) and np.isnan(text_value):
        # Handle NaN values if needed
        pass

    print(df.at[index, 'WRITING.TEXT.Element:Text'])

Yeah
Ok thanks  
It\xe2\x80\x99s not the it\xe2\x80\x99s still dark
Give me one of your worsts then
 
Your ass too vanilla anyways
I\xe2\x80\x99m really really bored
Cause I\xe2\x80\x99m God
Sure
It\xe2\x80\x99s been up  
I\xe2\x80\x99m 19 do it
Very
In due time brother, in due time  
U really do
Making a post about u
It was a joke jeez
Ouch  
Like spare a whore a dollar will ya  
Thanks  
My What-The-Fuck-O-Meter\n\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\n\n-Jeffery Dahmer wasn\xe2\x80\x99t as bad as u\n\n-Honestly you deserve to be locked up\n\n-EW \n\n-Bitch clear your cache and history jesus\n\n-Not VPN worthy but ew\n\n-That\xe2\x80\x99s like what old people find obscene\n\n-My dog likes worse shit\n\n-Bitch that\xe2\x80\x99s nothing
Expecting so much hate for revealing my true self
It was like a milisecond reaction too like help
W

In [64]:
for i in range(59, 61):
    # read the csv file
    df = pd.read_csv(f'Dataset/subject_{i}.csv')

    #print(f'Processing subject_{i}.csv')
        
    # loop through each row in the csv file
    for index, row in df.iterrows():

        text_value = row['WRITING.TEXT.Element:Text']
        #print(text_value)
        print(text_value[2])
        if isinstance(text_value, str) and text_value[-1] == '`' :
            df.at[index, 'WRITING.TEXT.Element:Text'] = text_value[0:-1]
            print(f"Removed 'b`' and '`' from 'WRITING.TEXT.Element:Text' in subject_{i}.csv")
        elif isinstance(text_value, float) and np.isnan(text_value):
            # Handle NaN values if needed
            pass

    # Save the modified DataFrame
    df.to_csv(f'Dataset/subject_{i}.csv', index=False)

IndexError: string index out of range

In [47]:
# tell me the coloumn names in the csv file
df1 = pd.read_csv('Dataset/subject_59.csv')
df1.head()

,ID,WRITING.TITLE,WRITING.DATE,WRITING.INFO,WRITING.TEXT.Element:Text
0,eRisk2023-T3_Subject37,b'Parents with jobs that primarily deal with ...,b'2022-10-14 14:10:19',reddit post,''
1,eRisk2023-T3_Subject37,b' ',b'2022-09-23 03:48:27',reddit post,'It\xe2\x80\x99s a moldy geode'
2,eRisk2023-T3_Subject37,b' ',b'2022-09-10 04:37:37',reddit post,"'Hehe, great minds think alike.'"
3,eRisk2023-T3_Subject37,b' ',b'2022-09-08 14:38:07',reddit post,"'Go find your nearest Tsutaya Books, and see i..."
4,eRisk2023-T3_Subject37,b' ',b'2022-09-08 07:57:46',reddit post,'Tsutaya Bookstore in Tokyo! [They\xe2\x80\x99...
